In [1]:
import numpy as np
import math
import cv2

### Helper functions (defined before main code)

In [2]:
# apply gaussian smoothing to the image
def gaussian_smoothing(img, kernel_size, sigma):
    kernel_size = int(kernel_size)
    sigma = float(sigma)
    
    # generate kernel based on kernel size
    kernel = np.zeros((kernel_size, kernel_size))
    
    mid = (kernel_size - 1) // 2
    
    y, x = np.ogrid[float(-mid):float(mid+1),float(-mid):float(mid+1)]
    kernel_sum = 0
    
    # loop through the kernel
    for i in range(kernel_size):
        for j in range(kernel_size):
            # gaussian formula
            e = math.exp((-((x[0][j]**2)+(y[i][0]**2))/(2*(sigma**2))))
            kernel[i][j] = e*(1/(2*math.pi*(sigma**2)))
            kernel_sum = kernel_sum + kernel[i][j]
            
    # loop through the kernel and average it out
    for i in range(kernel_size):
        for j in range(kernel_size):
            kernel[i][j] = kernel[i][j]/kernel_sum
    
    # resize the image with padding and apply convolution
    # result will be the smoothed image
    img_pad = resize_with_pad(img, kernel_size)
    img_out = convolution(img_pad, kernel)
    
    return img_out

In [3]:
# convolution function
# used to convolve images according to the kernel specified
def convolution(img, kernel):
    filter_size = int(math.sqrt(kernel.size))
    
    # get image dimensions
    img_row, img_col = img.shape
    
    mid = (filter_size - 1) // 2
    
    r = img_row - 2*mid
    c = img_col - 2*mid
    
    # create empty numpy array for output
    img_out = np.zeros((r, c))
    
    # loop through the image and fill convolved values
    for i in range(r):
        for j in range(c):
            for k in range(filter_size):
                for l in range(filter_size):
                    img_out[i][j] = img_out[i][j] + (kernel[k][l] * img[i+k][j+l])
            if(img_out[i][j] < 0):
                img_out[i][j] = 0
            else:
                img_out[i][j] = int(img_out[i][j])
                
    return img_out

In [4]:
# function to resize image by padding on sides
def resize_with_pad(img, filter_size):
    # get image dimensions
    img_row, img_col = img.shape
    
    mid = (filter_size - 1) // 2
    
    # create empty output image
    img_out = np.zeros((img_row+2*mid, img_col+2*mid))
    
    # padding accross all sides
    img_out[mid:img_row+mid, mid:img_col+mid] = img
    img_out[:mid, :] = img_out[mid, :]
    img_out[-mid:, :] = img_out[-mid-1, :]
    img_out[:, :mid] = img_out[:, mid].reshape(-1, 1)
    img_out[:, -mid:] = img_out[:, -mid-1].reshape(-1, 1)
    
    return img_out

In [5]:
# function to calculate Ix
# gradient in x using sobel filter
def gradient_in_x(img):
    kernel = np.array([[-1,0,+1], [-1,0,+1], [-1,0,+1]])
    
    # resize with padding and apply convolution
    img_pad = resize_with_pad(img, 3)
    img_out = convolution(img_pad, kernel)
    
    return img_out

In [6]:
# function to calculate Iy
# same as above but different filter
def gradient_in_y(img):
    kernel = np.array([[-1,-1,-1], [0,0,0], [+1,+1,+1]])
    
    # resize with padding and apply convolution
    img_pad = resize_with_pad(img, 3)
    img_out = convolution(img_pad, kernel)
    
    return img_out

In [7]:
# function to calculate the second moment matrix, M
def get_M(img):
    # get the image as an array
    img_arr = np.asarray(img)
    
    # calculate Ix and Iy
    img_x = gradient_in_x(img_arr)
    img_y = gradient_in_y(img_arr)
    
    # image dimensions
    r, c = img_x.shape
    
    # arrays for Ixx, Iyy, and Ixy
    img_xx = np.zeros((r, c))
    img_yy = np.zeros((r, c))
    img_xy = np.zeros((r, c))
    
    # loop through image dimensions and fill the values for Ixx, Iyy, and Ixy
    for i in range(r):
        for j in range(c):
            img_xx[i][j] = img_x[i][j]*img_x[i][j]
            img_yy[i][j] = img_y[i][j]*img_y[i][j]
            img_xy[i][j] = img_x[i][j]*img_y[i][j]
    
    # get the second order derivatives by gaussian smoothing
    Ixx = gaussian_smoothing(img_xx, 5, 1)
    Iyy = gaussian_smoothing(img_yy, 5, 1)
    Ixy = gaussian_smoothing(img_xy, 5, 1)
    
    # second moment matrix
    M = [[[] for i in range(c)] for j in range(r)]
    
    # loop through M and fill in the above calculated values
    for i in range(r):
        for j in range(c):
            M[i][j] = [Ixx[i][j], Ixy[i][j], Iyy[i][j]]
            
    return M

In [8]:
# function to get the corner response function, R
def get_R(img, M, k, threshold):
    # get image as an array
    img_arr = np.asarray(img)
    
    # image dimensions
    r, c = img_arr.shape
    
    # corner response matrix
    R = np.zeros((r, c))
    
    # variables for corners and number of corners
    corners = []
    num_corners = 0
    
    # loop through R
    for i in range(r):
        for j in range(c):
            R[i][j] = ((M[i][j][0]*M[i][j][2]) - (M[i][j][1]**2)) - (k*((M[i][j][0]+M[i][j][2])**2))
            
    # resize R with padding and computer the new dimensions
    R = resize_with_pad(R, 3)
    R_row, R_col = R.shape
    
    # non-max suppression
    # discard corners with value greater than threshold
    for i in range(1, R_row-1):
        for j in range(1, R_col-1):
            if(R[i][j] > threshold):
                if(R[i][j] == max(R[i-1][j-1], R[i-1][j], R[i-1][j+1], R[i][j-1], R[i][j], R[i][j+1], R[i+1][j-1], R[i+1][j], R[i+1][j+1])):
                    corners.append([i-1, j-1, R[i-1][j-1]])
                    num_corners = num_corners + 1
                    
    return corners

In [9]:
# read images
rgb1 = cv2.imread("problem1/rgb1.png", cv2.IMREAD_GRAYSCALE)
rgb2 = cv2.imread("problem1/rgb2.png", cv2.IMREAD_GRAYSCALE)
rgb3 = cv2.imread("problem1/rgb3.png", cv2.IMREAD_GRAYSCALE)

# read depth maps
depth1 = cv2.imread("problem1/depth1.png", cv2.IMREAD_GRAYSCALE)
depth2 = cv2.imread("problem1/depth2.png", cv2.IMREAD_GRAYSCALE)
depth3 = cv2.imread("problem1/depth3.png", cv2.IMREAD_GRAYSCALE)

### Part 1

In [10]:
# Second moment matrix with corner response
rgb1_M = get_M(rgb1)
rgb1_corners = get_R(rgb1, rgb1_M, 0.05, 12000000)

rgb2_M = get_M(rgb2)
rgb2_corners = get_R(rgb2, rgb2_M, 0.05, 12000000)

rgb3_M = get_M(rgb3)
rgb3_corners = get_R(rgb3, rgb3_M, 0.05, 12000000)

In [11]:
print(len(rgb1_corners))
print(len(rgb2_corners))
print(len(rgb3_corners))

172
301
247


In [12]:
# get the top 100 corners with strongest response
rgb1_corners_top100 = sorted(rgb1_corners, reverse=True, key=lambda x: x[2])[:100]
rgb2_corners_top100 = sorted(rgb2_corners, reverse=True, key=lambda x: x[2])[:100]
rgb3_corners_top100 = sorted(rgb3_corners, reverse=True, key=lambda x: x[2])[:100]

### Part 2

In [13]:
K_inv = np.linalg.inv(np.array([[525.0, 0, 319.5], [0, 525.0, 239.5], [0, 0, 1]]))

In [14]:
# function to convert the generated corners into 3D points
def points_to_3d(corners, depth_map):
    pts_3d = []
    
    # corner has (x, y, corner response value)
    for corner in corners:
        x, y, _ = corner
        
        depth = depth_map[x,y]
        
        # discard points with zero depth
        if depth == 0:
            continue
        
        # convert the 2d points into homogeneous coordinates
        homogeneous = np.array([x, y, 1])
        
        # normalize the homogeneous coordinates
        normalized = K_inv.dot(homogeneous)
        
        # convert to world coordinate
        world = depth * normalized
        
        # scale the 3D point and add to the list
        point_3d = world / 5000
        pts_3d.append(point_3d)
        
    return pts_3d

In [15]:
rgb1_3d_pts = points_to_3d(rgb1_corners_top100, depth1)
rgb2_3d_pts = points_to_3d(rgb2_corners_top100, depth2)
rgb3_3d_pts = points_to_3d(rgb3_corners_top100, depth3)

### Part 3

In [16]:
# function to calculate the rank transform of image
# similar to the function I used in Assignment 2
def rank_transform(image):
    # specify window size
    window = 5
    
    mid = int((window - 1) / 2)
    
    # convert image to array
    arr = np.asarray(image)
    
    # get width and height of image
    width = arr.shape[0]
    height = arr.shape[1]
    
    # create empty array for output
    arr_out = np.zeros((width, height))
    
    # loop through entire image
    for i in range(width):
        for j in range(height):
            # loop through window
            for u in range(i-mid, i+mid+1):
                for v in range(j-mid, j+mid+1):
                    # condition to check pixel values
                    if (0<=u<width and 0<=v<height):
                        if (arr[u][v] < arr[i][j]):
                            arr_out[i][j] += 1
                            
    arr_out = arr_out.astype(np.uint16)
    #print(arr_out)
    
    return arr_out

In [17]:
# function to calculate SAD between the rank transformed images and corners
def sum_absdiff(rt1, rt2, corners1, corners2):
    # take rank transformed images as arrays
    rt1 = np.asarray(rt1)
    rt2 = np.asarray(rt2)
    
    # get dimensions
    r, c = rt1.shape
    
    # create distances array to store the relative distance
    distances = []
    
    # loop through the corners
    for cnr2 in corners2:
        for cnr1 in corners1:
            s = 0
            # check within 11x11 neighborhood
            for i in range(-5, 6):
                li = i + cnr2[0]
                ri = i + cnr1[0]
                for j in range(-5, 6):
                    lj = j + cnr2[1]
                    rj = j + cnr1[1]
                    if(0<=li<r and 0<=lj<c and 0<=ri<r and 0<=rj<c):
                        s += abs(int(rt2[li][lj]) - int(rt1[ri][rj]))
                        
            # store the points according to the distances
            distances.append([s, [cnr2[0], cnr2[1]], [cnr1[0], cnr1[1]]])
    
    # sort the distances in ascending order
    distances.sort()
    
    return distances

In [18]:
# get rank transform
rgb1_rt = rank_transform(rgb1)
rgb2_rt = rank_transform(rgb2)
rgb3_rt = rank_transform(rgb3)

In [19]:
rgb1_rt

array([[ 2,  8, 13, ...,  2,  1,  0],
       [ 2,  9, 17, ...,  9,  2,  2],
       [ 2,  8, 24, ..., 12,  9, 13],
       ...,
       [ 3,  5, 14, ..., 10,  4,  1],
       [10,  6, 17, ...,  7,  7,  2],
       [ 2,  3,  5, ...,  3,  3,  5]], dtype=uint16)

In [20]:
# perform corner matching using SAD
cm_21 = sum_absdiff(rgb2_rt, rgb1_rt, rgb2_corners_top100, rgb1_corners_top100)
cm_23 = sum_absdiff(rgb2_rt, rgb3_rt, rgb2_corners_top100, rgb3_corners_top100)

In [21]:
# get the top 10 3D points from the corner matching
# similar to points_to_3d() function
def points_to_3d_top10(corners, depth_map):
    pts_3d = []
    
    for corner in corners:
        x, y = corner
        
        depth = depth_map[x,y]
        
        # discard point with zero depth
        if depth == 0:
            #print(x,y)
            continue
        
        # convert 2d points to homogeneous coordinates
        homogeneous = np.array([x, y, 1])
        
        # normalize them
        normalized = K_inv.dot(homogeneous)
        
        # convert to world coordinate
        world = depth * normalized
        
        # and scale the points
        point_3d = world / 5000
        pts_3d.append(point_3d)
        
    return np.array(pts_3d[:10])

In [22]:
corners_21_rgb1 = []
corners_21_rgb2 = []
corners_23_rgb3 = []
corners_23_rgb2 = []

for i in range(len(cm_21)):
    corners_21_rgb1.append(cm_21[i][1])
    corners_21_rgb2.append(cm_21[i][2])

for i in range(len(cm_23)):
    corners_23_rgb3.append(cm_23[i][1])
    corners_23_rgb2.append(cm_23[i][2])

In [23]:
top10_21_rgb1 = points_to_3d_top10(corners_21_rgb1, depth1)
top10_21_rgb2 = points_to_3d_top10(corners_21_rgb2, depth2)
top10_23_rgb3 = points_to_3d_top10(corners_23_rgb3, depth3)
top10_23_rgb2 = points_to_3d_top10(corners_23_rgb2, depth2)

### Part 4

In [24]:
# RANSAC to get rigid transformations for image 1 and image 3
def RANSAC_3D_points(P1, P2, max_iterations=1000, threshold=0.1):
    # get number of points
    n = P2.shape[0]
    inliers = []
    
    # variables for best rotation and translation matrix
    best_R = None
    best_t = None
    best_num_inliers = 0
    
    # loop through number of iteratiosn
    for i in range(max_iterations):
        
        # randomly select 3 points and assign as the formula given in pdf
        idx = np.random.choice(n, 3, replace=False)
        P1_3 = P1[idx, :]
        P2_3 = P2[idx, :]
        
        # compute v1 and v2 vectors
        v1_1 = P1_3[0] - P1_3[1]
        v1_2 = P1_3[1] - P1_3[2]
        v2_1 = P2_3[0] - P2_3[1]
        v2_2 = P2_3[1] - P2_3[2]
        
        # estimate R and t from the v1 and v2 vectors
        # formula used as in pdf
        R1 = np.column_stack((v2_1, np.cross(v2_1, v2_2), v2_2))
        R1_pinv = np.linalg.pinv(R1)
        t1 = P2_3[0] - np.dot(R1, P1_3[0])
        R1_, S, V = np.linalg.svd(R1_pinv @ np.column_stack((v1_1, np.cross(v1_1, v1_2), v1_2)))
        R1 = R1_ @ V.T
        t1 = P2_3[0] - R1 @ P1_3[0]
        
        # calculate error for each correspondence
        P1_est = P1 @ R1.T + t1.T
        errors = np.linalg.norm(P2 - P1_est, axis=1)
        
        # select inliers
        inliers_idx = np.where(errors < threshold)[0]
        num_inliers = len(inliers_idx)
        
        # update best R, t and inliers if necessary
        if num_inliers > best_num_inliers:
            best_R = R1
            best_t = t1
            best_num_inliers = num_inliers
            inliers = inliers_idx
    
    return best_R, best_t

In [25]:
# get (R1, t1) and (R3, t3)
R1, t1 = RANSAC_3D_points(top10_21_rgb1, top10_21_rgb2)
R3, t3 = RANSAC_3D_points(top10_23_rgb3, top10_23_rgb2)

In [26]:
R1

array([[-0.5       ,  0.70710678, -0.5       ],
       [-0.70710678,  0.        ,  0.70710678],
       [ 0.5       ,  0.70710678,  0.5       ]])

In [27]:
t1

array([ 0.00053198, -0.00088804,  0.00160037])

In [28]:
R3

array([[ 0.46789077, -0.48104872, -0.74139757],
       [-0.71978115, -0.69419048, -0.00382996],
       [-0.51282874,  0.535436  , -0.67105512]])

In [29]:
t3

array([ 0.0041348 , -0.00135428,  0.0090061 ])

### Part 5

In [30]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [31]:
# read input images as color images
rgb1_color = cv2.imread("problem1/rgb1.png")
rgb1_color = cv2.cvtColor(rgb1_color, cv2.COLOR_BGR2RGB)

rgb2_color = cv2.imread("problem1/rgb2.png")
rgb2_color = cv2.cvtColor(rgb2_color, cv2.COLOR_BGR2RGB)

rgb3_color = cv2.imread("problem1/rgb3.png")
rgb3_color = cv2.cvtColor(rgb3_color, cv2.COLOR_BGR2RGB)

In [32]:
# function to convert all points in image to 3D
# similar to the above function 
def all_points_to_3d(depth_map):
    pts_3d = []
    
    for x in range(depth_map.shape[0]):
        for y in range(depth_map.shape[1]):
            d = depth_map[x,y]
            
            #if d == 0:
                #continue
            
            # convert all points to homogeneous
            homogeneous = np.array([x, y, 1])
            
            # normalize
            normalized = K_inv.dot(homogeneous)
            
            # convert to world coordinates
            world = d * normalized
            
            # scaling
            point_3d = world / 5000
            pts_3d.append(point_3d)
        
    return np.array(pts_3d)

In [33]:
# get all 3d points
all_rgb1 = all_points_to_3d(depth1)
all_rgb2 = all_points_to_3d(depth2)
all_rgb3 = all_points_to_3d(depth3)

In [34]:
# initialize color matrix
colors1 = rgb1_color.reshape((-1,3))
colors2 = rgb2_color.reshape((-1,3))
colors3 = rgb3_color.reshape((-1,3))

In [35]:
# combine the points with RGB color matrix
pts1_colored = np.hstack((all_rgb1, colors1))
pts2_colored = np.hstack((all_rgb2, colors2))
pts3_colored = np.hstack((all_rgb3, colors3))

In [36]:
# transform points in images 1 and 3 to the (R, t) calculated above
pts1_transformed = (R1.dot(pts1_colored[:, :3].T) + t1.reshape((3, 1))).T
pts3_transformed = (R3.dot(pts3_colored[:, :3].T) + t3.reshape((3, 1))).T

In [37]:
# add color matrix to points in images 1 and 3
pts1_transformed_with_color = np.hstack((pts1_transformed, np.zeros((pts1_transformed.shape[0], 3))))
pts3_transformed_with_color = np.hstack((pts3_transformed, np.zeros((pts3_transformed.shape[0], 3))))

In [38]:
# combine all points from images 1, 2, and 3
all_pts = np.vstack((pts1_transformed_with_color, pts2_colored, pts3_transformed_with_color))

In [39]:
# get all points and colors
all_colors = all_pts[:, 3:]
all_pts = all_pts[:, :3]

In [40]:
# generate point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(all_pts)
pcd.colors = o3d.utility.Vector3dVector(all_colors / 255)
o3d.io.write_point_cloud("merged_pcd.ply", pcd)

True

In [41]:
test_ply = o3d.io.read_point_cloud("merged_pcd.ply")
o3d.visualization.draw_geometries([test_ply])